# Exercise - Regression


The data set for this exercise includes information on house sales in King County, WA (between May 2014 and May 2015). (Each row in the data set pertains to one house. There is a total of 21,613 houses in the data set). Use this data set to predict the sale price of a house (i.e., the `price` column) based on the characteristics of the house. A model can be helpful for buyers, sellers, realtors, and lenders.

## Description of Variables

The description and type of each variable is provided in "KC house data - Data Dictionary.docx". Make sure to read this document to learn about the variables.

## Goal

Use the **kc_house_data.csv** data set and build a model to predict **price**. <br>

# Read and Prepare the Data

In [216]:
# Common imports

import pandas as pd
import numpy as np

np.random.seed(42)

# Get the data

In [217]:
#We will predict the "price" value in the data set:

housing = pd.read_csv("kc_house_data.csv")
housing.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,432000.0,5.0,2.75,2060.0,329903.0,1.5,0,3,5,7.0,2060,0,1989.0,0,98022.0,47.1776,-121.944,2240,220232.0
1,170000.0,2.0,1.00,810.0,8424.0,1.0,0,0,4,6.0,810,0,1959.0,0,98023.0,47.3286,-122.346,820,8424.0
2,235000.0,3.0,1.00,960.0,5030.0,1.0,0,0,3,7.0,960,0,1955.0,0,98118.0,47.5611,-122.280,1460,5400.0
3,350000.0,2.0,1.00,830.0,5100.0,1.0,0,0,4,7.0,830,0,1942.0,0,98126.0,47.5259,-122.379,1220,5100.0
4,397380.0,2.0,1.00,1030.0,5072.0,1.0,0,0,3,6.0,1030,0,1924.0,1958,98115.0,47.6962,-122.294,1220,6781.0


In [218]:
housing.shape

(21613, 19)

# Split data (train/test)

In [251]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(housing, test_size=0.3)

# Data Prep

Perform your data prep here. You can use pipelines like we do in the tutorials. Otherwise, feel free to use your own data prep steps. Eventually, you should do the following at a minimum:<br>
- Separate inputs from target<br>
- Impute/remove missing values<br>
- Standardize the continuous variables<br>
- One-hot encode categorical variables<br>

In [252]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the target variable 

In [253]:
train_y = train['price']
test_y = test['price']

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)

## Set the data type of "zipcode" to categorical

In [254]:
train_inputs['zipcode'] = train_inputs['zipcode'].astype('object')
test_inputs['zipcode'] = test_inputs['zipcode'].astype('object')

##  Identify the numeric, binary, and categorical columns

In [255]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [256]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['waterfront']

In [257]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [258]:
binary_columns

['waterfront']

In [259]:
numeric_columns

['bedrooms',
 'bathrooms',
 'sqft_living',
 'sqft_lot',
 'floors',
 'view',
 'condition',
 'grade',
 'sqft_above',
 'sqft_basement',
 'yr_built',
 'yr_renovated',
 'lat',
 'long',
 'sqft_living15',
 'sqft_lot15']

In [260]:
categorical_columns

['zipcode']

# Pipeline

In [261]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [262]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [263]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [264]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

In [265]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [266]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

<15129x88 sparse matrix of type '<class 'numpy.float64'>'
	with 257318 stored elements in Compressed Sparse Row format>

In [267]:
train_x.shape

(15129, 88)

# Tranform: transform() for TEST

In [268]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

<6484x88 sparse matrix of type '<class 'numpy.float64'>'
	with 110266 stored elements in Compressed Sparse Row format>

In [269]:
test_x.shape

(6484, 88)

# Calculate the Baseline

In [270]:
from sklearn.dummy import DummyRegressor

dummy_regr = DummyRegressor(strategy="mean")

dummy_regr.fit(train_x, train_y)

DummyRegressor()

In [271]:
from sklearn.metrics import mean_squared_error

In [272]:
dummy_train_pred = dummy_regr.predict(train_x)

baseline_train_mse = mean_squared_error(train_y, dummy_train_pred)

baseline_train_rmse = np.sqrt(baseline_train_mse)

print('Baseline Train RMSE: {}' .format(baseline_train_rmse))

Baseline Train RMSE: 367608.09196674253


In [273]:
dummy_test_pred = dummy_regr.predict(test_x)

baseline_test_mse = mean_squared_error (test_y, dummy_test_pred)

baseline_test_rmse = np.sqrt(baseline_test_mse)

print('Baseline Test RMSE: {}' .format(baseline_test_rmse))

Baseline Test RMSE: 366759.7466975898


# Train a SGD model (with no regularization)

In [274]:
from sklearn.linear_model import SGDRegressor 

# eta0 = learning rate
# penalty = regularization term
# max_iter = number of passes over training data (i.e., epochs)

sgd_reg = SGDRegressor(max_iter=100, penalty=None, eta0=0.01) 

sgd_reg.fit(train_x, train_y)



SGDRegressor(max_iter=100, penalty=None)

### Generate the error metrics

In [275]:
sgd_reg.predict(test_x)

array([171749.09848864, 526244.65597096, 489010.5392605 , ...,
       391224.94238512, 409283.86509481, 566101.58032502])

In [276]:

# Create a new DataFrame

predictions = pd.DataFrame(sgd_reg.predict(test_x), columns=['Predicted'])

predictions


,Predicted
0,171749.098489
1,526244.655971
2,489010.539260
3,807833.402509
4,192267.480175
...,...
6479,494120.033052
6480,748146.943245
6481,391224.942385
6482,409283.865095


In [277]:
# Add the actual to the same DataFrame

predictions['Actual'] = np.array(test_y)

predictions



,Predicted,Actual
0,171749.098489,225000.0
1,526244.655971,455950.0
2,489010.539260,470000.0
3,807833.402509,700000.0
4,192267.480175,279900.0
...,...,...
6479,494120.033052,407000.0
6480,748146.943245,659950.0
6481,391224.942385,476500.0
6482,409283.865095,257000.0


In [278]:
#Train RMSE
reg_train_pred = sgd_reg.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 163890.87190991268


In [279]:
#Test RMSE
reg_test_pred = sgd_reg.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 166087.73797465957


# Try L1 Regularization in SGD

In [280]:

#Stochastic Gradient:
sgd_reg_L1 = SGDRegressor(max_iter=100, penalty='l1', alpha = 0.1, eta0=0.01)

sgd_reg_L1.fit(train_x, train_y)


SGDRegressor(alpha=0.1, max_iter=100, penalty='l1')

### Generate the error metrics

In [281]:

#Train RMSE
reg_train_pred = sgd_reg_L1.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 163479.92930633939


In [282]:
#Test RMSE
reg_test_pred = sgd_reg_L1.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 165276.1380551687


# Try L2 Regularization in SGD

In [283]:

#Stochastic Gradient:

sgd_reg_L2 = SGDRegressor(max_iter=100, penalty='l2', alpha = 0.1, eta0=0.01)

sgd_reg_L2.fit(train_x, train_y)


SGDRegressor(alpha=0.1, max_iter=100)

### Generate the error metrics

In [284]:
#Train RMSE
reg_train_pred = sgd_reg_L2.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 202958.2672051712


In [285]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 202999.80019178765


# Try ElasticNet in SGD

In [286]:

#Stochastic Gradient:
sgd_reg_elastic = SGDRegressor(max_iter=100, penalty='elasticnet', l1_ratio=0.5, alpha = 0.1, 
                          eta0=0.01)
sgd_reg_elastic.fit(train_x, train_y)


SGDRegressor(alpha=0.1, l1_ratio=0.5, max_iter=100, penalty='elasticnet')

### Generate the error metrics

In [287]:
#Train RMSE
reg_train_pred = sgd_reg_elastic.predict(train_x)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 197480.97945412042


In [288]:
#Test RMSE
reg_test_pred = sgd_reg_elastic.predict(test_x)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 198204.4190484167


# Create Polynomial Features

Create polynomial features with degree = 2. 

In [289]:
from sklearn.preprocessing import PolynomialFeatures

# Create second degree terms and interaction terms
poly_features = PolynomialFeatures(degree=2).fit(train_x)

train_x_poly = poly_features.transform(train_x)

test_x_poly = poly_features.transform(test_x)

In [290]:
train_x_poly.shape

(15129, 4005)

In [291]:
test_x_poly.shape

(6484, 4005)

# Try L2 Regularization in SGD (with polynomial features)

In [298]:
sgd_reg_L2 = SGDRegressor(max_iter=100, penalty='l2', alpha = 0.1, eta0=0.01)

sgd_reg_L2.fit(train_x_poly, train_y)

SGDRegressor(alpha=0.1, max_iter=100)

### Generate the error metrics

In [299]:
reg_train_pred = sgd_reg_L2.predict(train_x_poly)

train_mse = mean_squared_error(train_y, reg_train_pred)

train_rmse = np.sqrt(train_mse)

print('Train RMSE: {}' .format(train_rmse))

Train RMSE: 1135592599764.2163


In [300]:
#Test RMSE
reg_test_pred = sgd_reg_L2.predict(test_x_poly)

test_mse = mean_squared_error (test_y, reg_test_pred)

test_rmse = np.sqrt(test_mse)

print('Test RMSE: {}' .format(test_rmse))

Test RMSE: 979507552321.4895
